In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import mutual_info_regression
from sklearn.feature_selection import mutual_info_classif
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

In [2]:
data = pd.read_csv('/Users/shanmugapriyan/Downloads/csv_preprocessed.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1215675 entries, 0 to 1215674
Data columns (total 63 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   Dur           1215675 non-null  float64
 1   sTos          1215675 non-null  float64
 2   dTos          1215675 non-null  float64
 3   sTtl          1215675 non-null  float64
 4   dTtl          1215675 non-null  float64
 5   sHops         1215675 non-null  float64
 6   dHops         1215675 non-null  float64
 7   TotPkts       1215675 non-null  float64
 8   SrcPkts       1215675 non-null  float64
 9   DstPkts       1215675 non-null  float64
 10  TotBytes      1215675 non-null  float64
 11  SrcBytes      1215675 non-null  float64
 12  DstBytes      1215675 non-null  float64
 13  Offset        1215675 non-null  float64
 14  sMeanPktSz    1215675 non-null  float64
 15  dMeanPktSz    1215675 non-null  float64
 16  Load          1215675 non-null  float64
 17  SrcLoad       1215675 non-n

In [4]:
X = data.drop(columns=["Attack Type"])

In [5]:
Y = data["Attack Type"]

In [6]:
def mutual_info(X, Y):
  mutual_info_arr = mutual_info_classif(X, Y)
  series_info = pd.Series(mutual_info_arr)
  series_info.index = X.columns
  series_top = series_info.sort_values(ascending=False)[:20]
  return series_top

In [7]:
# result = mutual_info(X, Y)
# result.plot.bar(figsize=(20,8))

In [8]:
# new_data = data[result.keys()]
# new_data.head()

In [9]:
# Hard coding now to skip the mutual information step
new_data = data[["SrcWin", "DstWin", "dHops", "dTtl", "TotBytes", "SrcBytes", "sMeanPktSz", "DstGap", "SrcGap", "dTos", "DstTCPBase", "SrcTCPBase", "TcpRtt", "Proto_udp", "DstBytes", "AckDat" , "dMeanPktSz", "Proto_tcp", "SynAck", "Load"]]
new_data.head()

,SrcWin,DstWin,dHops,dTtl,TotBytes,SrcBytes,sMeanPktSz,DstGap,SrcGap,dTos,DstTCPBase,SrcTCPBase,TcpRtt,Proto_udp,DstBytes,AckDat,dMeanPktSz,Proto_tcp,SynAck,Load
0,3.442533e-07,6.217791e-08,-0.973059,-0.00732,-0.117591,-0.099523,0.164660,-0.006334,-0.00383,-0.094739,-4.275809e-14,3.187841e-12,-0.267969,0.0,-0.066388,-0.388062,-0.287785,0.0,-0.044552,-0.008291
1,3.442533e-07,6.217791e-08,-0.973059,-0.00732,-0.117591,-0.099523,0.164660,-0.006334,-0.00383,-0.094739,-4.275809e-14,3.187841e-12,-0.267969,0.0,-0.066388,-0.388062,-0.287785,0.0,-0.044552,-0.008291
2,3.442533e-07,6.217791e-08,-0.973059,-0.00732,8.131316,9.877388,8.044846,-0.006334,-0.00383,-0.094739,-4.275809e-14,3.187841e-12,-0.267969,1.0,0.224388,-0.388062,0.972879,0.0,-0.044552,-0.007624
3,3.442533e-07,6.217791e-08,-0.973059,-0.00732,7.224216,8.734380,8.598644,-0.006334,-0.00383,-0.094739,-4.275809e-14,3.187841e-12,-0.267969,1.0,0.259298,-0.388062,0.922515,0.0,-0.044552,-0.007698
4,3.442533e-07,6.217791e-08,-0.973059,-0.00732,9.162383,11.165238,8.769752,-0.006334,-0.00383,-0.094739,-4.275809e-14,3.187841e-12,-0.267969,1.0,0.201274,-0.388062,0.811590,0.0,-0.044552,-0.007541


In [10]:
def concat_column_for_plot(pca_data, column_name):
  for_plot = pd.concat([pca_data, data[column_name]], axis = 1)
  return for_plot

In [11]:
new_data = concat_column_for_plot(new_data, "Attack Type")

In [12]:
X_train, X_test, y_train, y_test = train_test_split(new_data.loc[:, new_data.columns != 'Attack Type'], new_data['Attack Type'],
                                                    stratify=new_data['Attack Type'],
                                                    test_size=0.15)

X_train = pd.DataFrame(X_train, columns=new_data.columns.to_list()[:-1])
X_test = pd.DataFrame(X_test, columns=new_data.columns.to_list()[:-1])
y_train = pd.DataFrame(y_train, columns=['Attack Type'])
y_test = pd.DataFrame(y_test, columns=['Attack Type'])

print("Training dataset size:", X_train.shape)
print("Testing dataset size:", X_test.shape)
print("Training target size:", y_train.shape)
print("Testing target size:", y_test.shape)

Training dataset size: (1033323, 20)
Testing dataset size: (182352, 20)
Training target size: (1033323, 1)
Testing target size: (182352, 1)


In [13]:
def get_pca_df(scaled_data, no_of_components):

  pca = PCA(n_components=no_of_components)
  Principal_components=pca.fit_transform(scaled_data)
  column_names = ["PC "+str(i) for i in range(1, no_of_components+1)]
  pca_df = pd.DataFrame(data = Principal_components, columns = column_names)
  return pca_df, pca

In [14]:
X_train = X_train.to_numpy() if isinstance(X_train, pd.DataFrame) else X_train
# PCA to reduce dimensionality from 20 to 15
pca = PCA(n_components=15)
X_train_pca = pca.fit_transform(X_train)

In [15]:
pip install annoy

/Users/shanmugapriyan/.zshenv:1: permission denied: /Library/Java/JavaVirtualMachines/jdk1.8.0_341.jdk
Note: you may need to restart the kernel to use updated packages.


In [16]:
from annoy import AnnoyIndex
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [17]:
n_trees = 10  # Number of trees in the index
index_dims = X_train_pca.shape[1]  # Number of dimensions in your data
n_neighbors = 19  # Number of nearest neighbors to consider

# Build the Annoy index
annoy_index = AnnoyIndex(index_dims, 'euclidean')
for i, x in enumerate(X_train_pca):
    annoy_index.add_item(i, x)
annoy_index.build(n_trees)

True

In [18]:
X_test = X_test.to_numpy() if isinstance(X_test, pd.DataFrame) else X_test

# Transform test data using PCA
X_test_pca = pca.transform(X_test)

In [19]:
# Query the index for nearest neighbors for all test data points
nearest_neighbors = []
for x in X_test_pca:
    nearest_neighbor_indices = annoy_index.get_nns_by_vector(x, n_neighbors)
    nearest_neighbors.append(nearest_neighbor_indices)

In [20]:
len(nearest_neighbors) == len(X_test_pca)

True

In [21]:
# Using KNN on the candidate neighbors
knn_classifier = KNeighborsClassifier(n_neighbors=n_neighbors)
knn_classifier.fit(X_train_pca, y_train)

/opt/anaconda3/lib/python3.11/site-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsClassifier(n_neighbors=19)

In [22]:
import numpy as np  # Import NumPy library

#Convert nearest neighbor indices to actual data points from training set
nearest_neighbors_train = np.array([X_train_pca[indices] for indices in nearest_neighbors])


# # Stack the arrays along the last axis to ensure each row corresponds to a single data point
# nearest_neighbors_train = np.concatenate(nearest_neighbors_train)


In [23]:
# Ensure that nearest_neighbors_train has the same number of samples as X_test
nearest_neighbors_train.shape[0] == len(y_test)


True

In [24]:
# Reshape nearest_neighbors_train to have a shape of (n_samples, n_neighbors, n_features)
nearest_neighbors_train_reshaped = nearest_neighbors_train.reshape(-1, n_neighbors, X_train_pca.shape[1])

# Predict labels for all nearest neighbor data at once
# Reshape nearest_neighbors_train_reshaped to have a shape of (n_samples * n_neighbors, n_features)
# This flattens the array along the first axis
predicted_labels_flat = knn_classifier.predict(nearest_neighbors_train_reshaped.reshape(-1, X_train_pca.shape[1]))

# Reshape predicted labels to have a shape of (n_samples, n_neighbors)
# This restores the original shape of the nearest_neighbors_train array
predicted_labels_reshaped = predicted_labels_flat.reshape(-1, n_neighbors)


In [25]:
# Now, for each test data point, you can aggregate the predictions from its nearest neighbors
# For example, you can use majority voting:
import scipy.stats
predicted_labels_majority = scipy.stats.mode(predicted_labels_reshaped, axis=1).mode.flatten()

# Evaluate accuracy
accuracy = accuracy_score(y_test, predicted_labels_majority)
print("Accuracy:", accuracy)

Accuracy: 0.9835702377818725


In [26]:
accuracy = accuracy_score(y_test, predicted_labels_majority)
print("Accuracy:", accuracy)

Accuracy: 0.9835702377818725


In [27]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []
Y = new_data['Attack Type']
X_train, pca = get_pca_df(X_train, 15)
for train_index, test_index in skf.split(X_train, y_train):
    X1_train, X1_test = X_train.iloc[train_index], X_train.iloc[test_index]
    y1_train, y1_test = y_train.iloc[train_index], y_train.iloc[test_index]
    # Initialize kNN classifier
    knn = KNeighborsClassifier(n_neighbors=19)
    # Train the classifier
    knn.fit(X1_train, y1_train)
    # Predict on the test set
    y_pred = knn.predict(X1_test)
    # Calculate evaluation metrics and store them
    accuracy_scores.append(accuracy_score(y1_test, y_pred))
    precision_scores.append(precision_score(y1_test, y_pred, average='weighted'))
    recall_scores.append(recall_score(y1_test, y_pred, average='weighted'))
    f1_scores.append(f1_score(y1_test, y_pred, average='weighted'))

/opt/anaconda3/lib/python3.11/site-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [ ]:
x_test = pca.transform(X_test)

In [ ]:
x_test

In [ ]:
y1_pred = knn.predict(x_test)

In [ ]:
accuracy_score(y_test, y1_pred)

In [ ]:
accuracy_scores